### 导入数据

In [7]:
import pandas as pd
import numpy as np
import os 
# path='/Users/chizhu/data/competition_data/快手活跃用户.tar/'
path='/mnt/datasets/fusai/'
os.listdir(path)

['video_create_log.txt',
 'user_activity_log.txt',
 'user_register_log.txt',
 'app_launch_log.txt']

In [8]:
action=pd.read_csv(path+"user_activity_log.txt",sep="\t",header=None,dtype={0: np.uint32, 1: np.uint8, 2: np.uint8, 3: np.uint32, 4: np.uint32, 5: np.uint8})
action.columns=['user_id','day','page','video_id','author_id',"action_type"]

register=pd.read_csv(path+"user_register_log.txt",sep="\t",header=None,dtype={0: np.uint32, 1: np.uint8, 2: np.uint8, 3: np.uint16})
register.columns=['user_id','register_day','register_type',"device_type"]

launch=pd.read_csv(path+"app_launch_log.txt",sep="\t",header=None,dtype={0: np.uint32, 1: np.uint8})
launch.columns=['user_id','day']

video=pd.read_csv(path+"video_create_log.txt",sep="\t",header=None,dtype={0: np.uint32, 1: np.uint8})
video.columns=['user_id','day']
action=action.sort_values(by=['user_id',"day"])
launch=launch.sort_values(by=['user_id',"day"])
video=video.sort_values(by=['user_id',"day"])
register=register.sort_values(by=['user_id',"register_day"])

In [34]:
try:
    from joblib import Parallel, delayed
except ImportError:
    import os
    os.system('pip install joblib')
def applyParallel( func,df,pred_firstday):
    retLst=Parallel(n_jobs=-1)(delayed(func)(df[df.user_id==userid],pred_firstday) for userid in df.user_id.unique())
    return pd.concat(retLst, axis=0)

In [35]:
def get_author_num(df):
    return len(df['user_id'].unique())
def get_author_user_count_feat(action,train_begin,register_lastday):
    action = action[(action.day>=train_begin)&(action.day<=register_lastday)]
    a=action.drop_duplicates(['user_id','author_id'])
    b=action.groupby("author_id",as_index=False).agg(get_author_num)[['author_id',"user_id"]]
    b.columns=['author_id',"user_count"]
    c=pd.merge(a,b,on="author_id",how="left")
    res=c.groupby("user_id",as_index=False)['user_count'].agg({"user_count_max":"max","user_count_min":"min",
    "user_count_mean":"mean","user_count_skew":"skew","user_count_std":"std"
})
    
    return res

In [36]:
%%time
data3_author=get_author_user_count_feat(action,15,30)

In [37]:
data3_author.to_csv("data3_author.csv",index=False)

### 特征提取

In [4]:

import datetime
import warnings
warnings.filterwarnings('ignore')

def get_action_type_feat(df,i):
    tm=pd.DataFrame(index=range(1))
    df.index=range(len(df))
    a=df[df.action_type==i]
    a['index']=a.index
    if len(a)<=1:
        tm['action_type_'+str(i)+"_day_dis_max"]=np.nan
        tm['action_type_'+str(i)+"_day_dis_min"]=np.nan
        tm['action_type_'+str(i)+"_day_dis_mean"]=np.nan
        tm['action_type_'+str(i)+"_day_dis_median"]=np.nan
        tm['action_type_'+str(i)+"_day_dis_std"]=np.nan
        tm['action_type_'+str(i)+"_day_dis_skew"]=np.nan
        tm['action_type_'+str(i)+"_day_dis_kurt"]=np.nan
        tm['action_type_'+str(i)+"_day_dis_last"]=np.nan
        ###
        tm['action_type_'+str(i)+"_dis_max"]=np.nan
        tm['action_type_'+str(i)+"_dis_min"]=np.nan
        tm['action_type_'+str(i)+"_dis_mean"]=np.nan
        tm['action_type_'+str(i)+"_dis_median"]=np.nan
        tm['action_type_'+str(i)+"_dis_std"]=np.nan
        tm['action_type_'+str(i)+"_dis_skew"]=np.nan
        tm['action_type_'+str(i)+"_dis_kurt"]=np.nan
        tm['action_type_'+str(i)+"_dis_last"]=np.nan
    else:
        a_diff=a.diff(1)
        tm['action_type_'+str(i)+"_day_dis_max"]=a_diff['day'].max()
        tm['action_type_'+str(i)+"_day_dis_min"]=a_diff['day'].min()
        tm['action_type_'+str(i)+"_day_dis_mean"]=a_diff['day'].mean()
        tm['action_type_'+str(i)+"_day_dis_median"]=a_diff['day'].median()
        tm['action_type_'+str(i)+"_day_dis_std"]=a_diff['day'].std()
        tm['action_type_'+str(i)+"_day_dis_skew"]=a_diff['day'].skew()
        tm['action_type_'+str(i)+"_day_dis_kurt"]=a_diff['day'].kurt()
        tm['action_type_'+str(i)+"_day_dis_last"]=a_diff['day'].values[-1]
        #####
        tm['action_type_'+str(i)+"_dis_max"]=a_diff['index'].max()
        tm['action_type_'+str(i)+"_dis_min"]=a_diff['index'].min()
        tm['action_type_'+str(i)+"_dis_mean"]=a_diff['index'].mean()
        tm['action_type_'+str(i)+"_dis_median"]=a_diff['index'].median()
        tm['action_type_'+str(i)+"_dis_std"]=a_diff['index'].std()
        tm['action_type_'+str(i)+"_dis_skew"]=a_diff['index'].skew()
        tm['action_type_'+str(i)+"_dis_kurt"]=a_diff['index'].kurt()
        tm['action_type_'+str(i)+"_dis_last"]=a_diff['index'].values[-1]
        
    return tm
        

def get_inpred_count(temp,flag):
    count=0
    for i in [flag+'_lastdaytopred+day_dis_mean',flag+'_lastdaytopred+day_dis_max',flag+"_lastdaytopred+day_dis_min",
    flag+"_lastdaytopred+day_dis_median"]:
        if temp[i].values[0]>0 and temp[i].values[0]<=7:
            count+=1
    return count
def get_launch_feat(df,pred_firstday):
    temp=pd.DataFrame(index=range(1))
    temp['user_id']=df['user_id'].unique()[0]
    temp['launch_count']=len(df)
    temp['launch_day_mean']=df['dur_day'].mean()
    temp['launch_day_median']=df['dur_day'].median()
    temp['launch_day_min']=df['dur_day'].min()
    temp['launch_day_max']=df['dur_day'].max()
    temp['launch_day_max-min']=temp['launch_day_max']-temp['launch_day_min']
    last_7days_len=len(df[df['dur_day']<=7])
    temp['last_7day_launch_rate']=last_7days_len/temp['launch_count']
    last_5days_len=len(df[df['dur_day']<=5])
    temp['last_5day_launch_rate']=last_5days_len/temp['launch_count']
    last_4days_len=len(df[df['dur_day']<=4])
    temp['last_4day_launch_rate']=last_4days_len/temp['launch_count']
    last_3days_len=len(df[df['dur_day']<=3])
    temp['last_3day_launch_rate']=last_3days_len/temp['launch_count']
    last_2days_len=len(df[df['dur_day']<=2])
    
    temp['last_7day_launch_cnt']=last_7days_len/7
    temp['last_5day_launch_cnt']=last_5days_len/5
    temp['last_4day_launch_cnt']=last_4days_len/4
    temp['last_3day_launch_cnt']=last_3days_len/3
    temp['last_2day_launch_cnt']=last_2days_len/2
    
    temp['launch_day_std']=df['dur_day'].std()
    temp['launch_day_skew']=df['dur_day'].skew()
    temp['launch_day_kurt']=df['dur_day'].kurt()
    # temp['last_launch_day_before_pred']=pred_firstday-df['day'].values[-1]
    df_diff=df.diff(1)
    temp['launch_day_dis_max']=df_diff['day'].max()
    temp['launch_day_dis_min']=df_diff['day'].min()
    temp['launch_day_dis_mean']=df_diff['day'].mean()
    temp['launch_day_dis_median']=df_diff['day'].median()
    temp['launch_day_dis_std']=df_diff['day'].std()
    temp['launch_day_dis_skew']=df_diff['day'].skew()
    temp['launch_day_dis_kurt']=df_diff['day'].kurt()
    temp['launch_day_dis_last']=df_diff['day'].values[-1]
     
    temp['launch_lastdaytopred+day_dis_mean']=0-temp['launch_day_min'].values[0]+temp['launch_day_dis_mean'].values[0]
    
    temp['launch_lastdaytopred+day_dis_max']=0-temp['launch_day_min'].values[0]+temp['launch_day_dis_max'].values[0]
    temp['launch_lastdaytopred+day_dis_min']=0-temp['launch_day_min'].values[0]+temp['launch_day_dis_min'].values[0]
    temp['launch_lastdaytopred+day_dis_median']=0-temp['launch_day_min'].values[0]+temp['launch_day_dis_median'].values[0]
    temp["launch_inpredday_count"]=get_inpred_count(temp,"launch")
    temp['launch_sum']=0
    for i in range(pred_firstday-7,pred_firstday):
        if i in df['day'].values:
            temp['last_'+str(pred_firstday-i)+"_launch_num"]=1
        else :
            temp['last_'+str(pred_firstday-i)+"_launch_num"]=0
        temp['launch_sum']+=temp['last_'+str(pred_firstday-i)+"_launch_num"]/(pred_firstday-i)
    for i in [2,3,4,5,6,7]:
        temp["launch_"+str(i)+"_continuous_day_count"]=get_continuous_day_count(i,df)
    
    return temp


def get_continuous_day_count(cont_num,df):
    day_set=df['day'].unique()

    day_count=0
    for i in day_set:
        flag=0
        for j in range(1,cont_num):
            if i+j in day_set:
                flag+=1
       
        if flag==cont_num-1:
            day_count+=1
    
    
    return day_count
            
    
    
def get_video_feat(df,pred_firstday):
    temp=pd.DataFrame(index=range(1))
    temp['user_id']=df['user_id'].unique()[0]
    temp['video_count']=len(df)
    
    temp['day_video_mean']=df.groupby('day',as_index=False).count()['user_id'].mean()
    temp['day_video_median']=df.groupby('day',as_index=False).count()['user_id'].median()
    temp['day_video_std']=df.groupby('day',as_index=False).count()['user_id'].std()
    temp['day_video_max']=df.groupby('day',as_index=False).count()['user_id'].max()
    temp['day_video_min']=df.groupby('day',as_index=False).count()['user_id'].min()
    temp['day_video_skew']=df.groupby('day',as_index=False).count()['user_id'].skew()
    temp['day_video_kurt']=df.groupby('day',as_index=False).count()['user_id'].kurt()
    temp['day_video_last']=df.groupby('day',as_index=False).count()['user_id'].values[-1]
    b=dict(zip(df.groupby('day',as_index=False).count()['day'].values,df.groupby('day',as_index=False).count()['user_id'].values))
    temp['video_sum']=0
    for i in range(pred_firstday-7,pred_firstday):
        if i in b.keys():
            temp['last_'+str(pred_firstday-i)+"_video_num"]=b[i]
        else :
            temp['last_'+str(pred_firstday-i)+"_video_num"]=0
        temp['video_sum']+=temp['last_'+str(pred_firstday-i)+"_video_num"]/(pred_firstday-i)
    temp['video_day_count']=df.groupby('day',as_index=False).count()['user_id'].count()
    df_temp=df.groupby('day',as_index=False).count()
    df_temp['dur_day']=df_temp['day'].apply(lambda x:pred_firstday-x,1)
    
    temp['video_day_mean']=df_temp['dur_day'].mean()
    temp['video_day_median']=df_temp['dur_day'].median()
    temp['video_day_min']=df_temp['dur_day'].min()
    temp['video_day_max']=df_temp['dur_day'].max()
    temp['video_day_max-min']=temp['video_day_max']-temp['video_day_min']
    last_7days_len=len(df_temp[df_temp['dur_day']<=7])
    temp['last_7day_video_rate']=last_7days_len/len(df_temp)
    
    last_5days_len=len(df_temp[df_temp['dur_day']<=5])
    temp['last_5day_video_rate']=last_5days_len/len(df_temp)
    last_4days_len=len(df_temp[df_temp['dur_day']<=4])
    temp['last_4day_video_rate']=last_4days_len/len(df_temp)
    last_3days_len=len(df_temp[df_temp['dur_day']<=3])
    temp['last_3day_video_rate']=last_3days_len/len(df_temp)
    last_2days_len=len(df_temp[df_temp['dur_day']<=2])
    
    temp['last_7day_video_cnt']=last_7days_len/7
    temp['last_5day_video_cnt']=last_5days_len/5
    temp['last_4day_video_cnt']=last_4days_len/4
    temp['last_3day_video_cnt']=last_3days_len/3
    temp['last_2day_video_cnt']=last_2days_len/2
   
    
    temp['video_day_std']=df_temp['dur_day'].std()
    temp['video_day_skew']=df_temp['dur_day'].skew()
    temp['video_day_kurt']=df_temp['dur_day'].kurt()
    # temp['last_video_day_before_pred']=pred_firstday-df_temp['day'].values[-1]
    df_diff=df_temp.diff(1)
    temp['video_day_dis_max']=df_diff['day'].max()
    temp['video_day_dis_min']=df_diff['day'].min()
    temp['video_day_dis_mean']=df_diff['day'].mean()
    temp['video_day_dis_median']=df_diff['day'].median()
    temp['video_day_dis_std']=df_diff['day'].std()
    temp['video_day_dis_skew']=df_diff['day'].skew()
    temp['video_day_dis_kurt']=df_diff['day'].kurt()
    temp['video_day_dis_last']=df_diff['day'].values[-1]
    
    
    for i in [2,3,4,5,6,7]:
        temp["video_"+str(i)+"_continuous_day_count"]=get_continuous_day_count(i,df_temp)
    
    return temp
  
    
    
def get_action_feat(df,pred_firstday):
    temp=pd.DataFrame(index=range(1))
    temp['user_id']=df['user_id'].values[0]
    temp['action_count']=len(df)
    
    t1=df.groupby('day',as_index=False).count()
    temp['day_action_mean']=t1['user_id'].mean()
    temp['day_action_median']=t1['user_id'].median()
    temp['day_action_std']=t1['user_id'].std()
    temp['day_action_max']=t1['user_id'].max()
    temp['day_action_min']=t1['user_id'].min()
    temp['day_action_skew']=t1['user_id'].skew()
    temp['day_action_kurt']=t1['user_id'].kurt()
    temp['day_action_last']=t1['user_id'].values[-1]
   
    ####author
   
    t2=df.groupby("author_id",as_index=False).count()
    temp['author_min']=t2['user_id'].min()
    temp['author_max']=t2['user_id'].max()
    temp['author_mean']=t2['user_id'].mean()
    temp['author_median']=t2['user_id'].median()
    temp['author_std']=t2['user_id'].std()
    temp['author_skew']=t2['user_id'].skew()
    temp['author_kurt']=t2['user_id'].kurt()
    temp['author_visit_cnt']=len(df['author_id'].unique())/temp['action_count']
    
    b=dict(zip(t1['day'].values,t1['user_id'].values))
    temp['action_sum']=0
    for i in range(pred_firstday-7,pred_firstday):
        if i in b.keys():
            temp['last_'+str(pred_firstday-i)+"_action_num"]=b[i]
        else :
            temp['last_'+str(pred_firstday-i)+"_action_num"]=0
        temp['action_sum']+=temp['last_'+str(pred_firstday-i)+"_action_num"]/(pred_firstday-i)
    temp['action_day_count']=len(t1)
    
    df_temp=t1
    df_temp['dur_day']=df_temp['day'].apply(lambda x:pred_firstday-x,1)

    temp['action_day_mean']=df_temp['dur_day'].mean()
    temp['action_day_median']=df_temp['dur_day'].median()
    temp['action_day_min']=df_temp['dur_day'].min()
    temp['action_day_max']=df_temp['dur_day'].max()
    temp['action_day_max-min']=temp['action_day_max']-temp['action_day_min']
    last_7days_len=len(df_temp[df_temp['dur_day']<=7])
    temp['last_7day_action_rate']=last_7days_len/temp['action_day_count']
    
    last_5days_len=len(df_temp[df_temp['dur_day']<=5])
    temp['last_5day_action_rate']=last_5days_len/temp['action_day_count']
    last_4days_len=len(df_temp[df_temp['dur_day']<=4])
    temp['last_4day_action_rate']=last_4days_len/temp['action_day_count']
    last_3days_len=len(df_temp[df_temp['dur_day']<=3])
    temp['last_3day_action_rate']=last_3days_len/temp['action_day_count']
    last_2days_len=len(df_temp[df_temp['dur_day']<=2])
#     temp['last_2day_action_rate']=last_2days_len/len(df)
#     last_1days_len=len(df_temp[df_temp['dur_day']<=1])
#     temp['last_1day_action_rate']=last_1days_len/len(df)

    temp['last_7day_action_cnt']=last_7days_len/7
    temp['last_5day_action_cnt']=last_5days_len/5
    temp['last_4day_action_cnt']=last_4days_len/4
    temp['last_3day_action_cnt']=last_3days_len/3
    temp['last_2day_action_cnt']=last_2days_len/2
    
    temp['action_day_std']=df_temp['dur_day'].std()
    temp['action_day_skew']=df_temp['dur_day'].skew()
    temp['action_day_kurt']=df_temp['dur_day'].kurt()
    # temp['last_action_day_before_pred']=pred_firstday-df_temp['day'].values[-1]
  
    starttime = datetime.datetime.now()
    df_diff=df_temp.diff(1)
    temp['action_day_dis_max']=df_diff['day'].max()
    temp['action_day_dis_min']=df_diff['day'].min()
    temp['action_day_dis_mean']=df_diff['day'].mean()
    temp['action_day_dis_median']=df_diff['day'].median()
    temp['action_day_dis_std']=df_diff['day'].std()
    temp['action_day_dis_skew']=df_diff['day'].skew()
    temp['action_day_dis_last']=df_diff['day'].values[-1]
    
    temp['action_lastdaytopred+day_dis_mean']=0-temp['action_day_min'].values[0]+temp['action_day_dis_mean'].values[0]
    
    temp['action_lastdaytopred+day_dis_max']=0-temp['action_day_min'].values[0]+temp['action_day_dis_max'].values[0]
    temp['action_lastdaytopred+day_dis_min']=0-temp['action_day_min'].values[0]+temp['action_day_dis_min'].values[0]
    temp['action_lastdaytopred+day_dis_median']=0-temp['action_day_min'].values[0]+temp['action_day_dis_median'].values[0]
    temp["action_inpredday_count"]=get_inpred_count(temp,"action")
   
    for i in [2,3,4,5,6,7]:
        temp["action_"+str(i)+"_continuous_day_count"]=get_continuous_day_count(i,df_temp)
   
    ###page
    temp['page_type_count']=len(df.page.unique())
    for i in range(5):
        temp['page_'+str(i)+"_count"]=len(df[df.page==i])/temp['action_count']
    ###action_type
    temp['action_type_count']=len(df.action_type.unique())
    for i in range(6):
        temp['action_type_'+str(i)+"_count"]=len(df[df.action_type==i])/temp['action_count']
   
    #####action_type_other
    for i in range(1,6):
        temp_other=get_action_type_feat(df,i)
        temp=temp.join(temp_other)
        
    df['id_same']=df["user_id"]-df['author_id']
    a=len(df[df['id_same']==0])
    temp['author_and_id_the_same']=a/temp['action_count']
    
    
    return temp   



def get_action_author_feat(action,register):
    author_temp=action.groupby(['author_id',"user_id"],as_index=False).count()
    author=author_temp.groupby("author_id",as_index=False).count()[['author_id','user_id']]
    author.rename(columns={'user_id':'author_count', 'author_id':'user_id', }, inplace = True)
    id_list=register[['user_id']]
    action_other_feat=pd.merge(id_list,author,on="user_id",how="left")
    
    star=action[action.action_type==2]
    star=star[['author_id','action_type']]
    star.columns=['user_id','followers']
    star_count=star.groupby("user_id",as_index=False).count()
    action_other_feat=pd.merge(action_other_feat,star_count,on="user_id",how="left")  
       
    
    return action_other_feat


def get_label(data,launch,pred_begin=None,pred_end=None):
    print("get label...")
    starttime = datetime.datetime.now()
    launch_data=launch[(launch.day>=pred_begin)&(launch.day<=pred_end)]
    target=launch_data.groupby("user_id",as_index=False).count()[['user_id']]
    target['label']=1
    train=pd.merge(data,target,on="user_id",how="left")
    train['label']=train['label'].fillna(0)
    train['label']=train['label'].astype(int)
    
    print("use time:",datetime.datetime.now()-starttime )
   
    
    return train

def get_data(action,video,register,launch,register_lastday,train_begin,pred_firstday):
    register_data=register[register.register_day<=register_lastday]
    launch_data=launch[(launch.day>=train_begin)&(launch.day<=register_lastday)]
    video_data=video[(video.day>=train_begin)&(video.day<=register_lastday)]
    action_data = action[(action.day>=train_begin)&(action.day<=register_lastday)]
    
    register_data['register_to_predday']=register_data.apply(lambda x:pred_firstday-x['register_day'],1)
    
    print("process launch...")
    starttime = datetime.datetime.now()
    launch_data['dur_day']=launch_data['day'].apply(lambda x:pred_firstday-x,1)
#     launch_feat=launch_data.groupby("user_id").apply(lambda x:get_launch_feat(x,pred_firstday))
#     launch_feat.index=range(len(launch_feat))
    launch_feat=applyParallel(get_launch_feat,launch_data,pred_firstday)
    print("use time:",datetime.datetime.now()-starttime )
    
    print("process video...")
    starttime = datetime.datetime.now()
#     video_feat=video_data.groupby("user_id").apply(lambda x:get_video_feat(x,pred_firstday))
#     video_feat.index=range(len(video_feat))
    video_feat=applyParallel(get_video_feat,video_data,pred_firstday)
    print("use time:",datetime.datetime.now()-starttime )
    
    print("process action...")
    starttime = datetime.datetime.now()
    # action_feat=action_data.groupby("user_id").apply(lambda x:get_action_feat(x,pred_firstday))
    # action_feat.index=range(len(action_feat))
    action_feat=applyParallel(get_action_feat,action_data,pred_firstday)
    print("use time:",datetime.datetime.now()-starttime )
    
    print("process action other...")
    starttime = datetime.datetime.now()
    action_other=get_action_author_feat(action_data,register_data)
    print("use time:",datetime.datetime.now()-starttime )
    
    ####merge
    
    data=pd.merge(register_data,launch_feat,on="user_id",how="left")
    data=pd.merge(data,video_feat,on="user_id",how="left")
    data=pd.merge(data,action_feat,on="user_id",how="left")
    data=pd.merge(data,action_other,on="user_id",how="left")
    
    
    
    return data
    
    

### 数据集的构建

In [5]:
import numpy as np
def get_more_fast(train_data):
    for i in train_data.columns:
        if type(train_data[i][0])==np.int64:
            train_data[i] = train_data[i].astype(np.int32)
        elif type(train_data[i][0])==np.float64:
            train_data[i] = train_data[i].astype(np.float32)
    return train_data

In [58]:
%%time
####data1
data1=get_data(action,video,register,launch,16,1,17)
data1=get_label(data1,launch,17,23)

In [61]:
data1=get_more_fast(data1)
data1.to_csv("data1_new2.csv",index=False)

In [59]:
%%time
####data2
data2=get_data(action,video,register,launch,23,8,24)
data2=get_label(data2,launch,24,30)

In [60]:
data2=get_more_fast(data2)
data2.to_csv("data2_new2.csv",index=False)

In [6]:
%%time
####data3
data3=get_data(action,video,register,launch,30,15,31)

In [7]:
data3=get_more_fast(data3)
data3.to_csv("/home/kesci/input/data3_new2.csv",index=False)

In [252]:
######压缩与解压

# !tar -jcvf  lgb.tar.bz2 /home/kesci/input


#### NOTE
* lgb+all_feat+leaves_36+feat0.5+nround_900 =>0.91013317
* lgb+all_feat+leaves_111+feat0.5+nround_900 => 0.91024836
* lgb+all_feat+leaves_132+feat0.5+nround_1000 => 0.91027567
* lgb+all_feat+leaves_132+feat0.5+nround_1500 => 0.91021246
* lgb+all_feat+leaves_132+feat0.5+nround_1000  * 0.57+ xgb+all_feat+depth_8+feat_0.5+nround_1100 *0.43 =>0.91035289
* lgb+all_feat+leaves_132+feat0.5+nround_1000 only use data2 for train =>0.90841265
* xgb+all_feat+depth_8+feat_0.5+nround_1000  =>0.91034
* lgb+new_feat.drop("register_day") use top126+leaves_32+feat_0.4+nround_1300 =>0.910638 lgb.txt
* lgb+new_feat.drop("register_day"+last_action/video/launch_to_pred) use top120+leaves_32+feat_0.4+nround_1200 =>0.9105 lgb_120.txt 
* lgb+new_feat.drop("register_day"+last_action/video/launch_to_pred) use top 128+leaves_127+feat_0.4+nround_850=>0.91078  lgb_128.txt
* xgb+new_feat.drop("register_day"+last_action/video/launch_to_pred) use top 128+depth_8+feat_0.7+nround_1000 =>0.91086 xgb_128.txt
* xgb+new_feat.drop("register_day"+last_action/video/launch_to_pred) use top 127+depth_8+feat_0.7+nround_940 =>0.91083 xgb_127.txt
#### B
* xgb +106feat+ depth_8+feat_0.7+nround_1200 =>0.91195 xgb_06.txt
* lgb 128feat+leaves_127+feat_0.7+bagging_0.7+bagging_freq_3+nround_1200+ xgb +106feat+ depth_8+feat_0.7+nround_1200 mean=>0.91197 bagging.txt
* lgb 98feat+leaves_127+feat_0.7+bagging_0.7+bagging_freq_3+nround_1400=>0.91191 lgb_98.txt

In [3]:
%%bash
tar -jxvf  data.tar.bz2 -C /home/kesci/

In [4]:
ls /home/kesci/home/kesci/input

In [5]:
#####load data
import pandas as pd
print("loading data...")
data1=pd.read_csv("/home/kesci/home/kesci/input/data1_new.csv")
data2=pd.read_csv("/home/kesci/home/kesci/input/data2_new.csv")
data3=pd.read_csv("/home/kesci/home/kesci/input/data3_new.csv")

In [136]:
data1=pd.merge(data1,data1_author[['user_id',"launch_day_dis_last",]],on="user_id",how="left")
data2=pd.merge(data2,data2_author[['user_id',"launch_day_dis_last",]],on="user_id",how="left")
data3=pd.merge(data3,data3_author[['user_id',"launch_day_dis_last",]],on="user_id",how="left")


In [135]:
data1_author=pd.read_csv("/home/kesci/home/kesci/input/data1_new2.csv")
data2_author=pd.read_csv("/home/kesci/home/kesci/input/data2_new2.csv")
data3_author=pd.read_csv("/home/kesci/home/kesci/input/data3_new2.csv")

In [6]:
data12=pd.concat([data1,data2])

In [7]:
train=data12
test=data3

In [27]:
import pandas as pd
result=pd.read_csv("0.6+0.4.txt",header=None)
result.columns=['user_id',"pred"]

In [28]:
r=register[register.register_day==30]
a30=action[action.day==30][['user_id',"day"]]
a30.columns=['user_id',"day30"]
a30=a30.drop_duplicates("user_id")
c=pd.merge(r,a30,on="user_id",how="left")
d=c[c.day30.notnull()]
e=pd.merge(d,result,on="user_id",how="left")

In [29]:
e=e.sort_values(by=["pred"],ascending=False)

In [30]:
f=e.tail(6000)

In [31]:
need=f.head(3000)[['user_id',"pred"]]
id=list(need.user_id.values)
result.set_index("user_id",inplace=True)
result.drop(id,0,inplace=True)
result=result.reset_index()


In [32]:
add=need

In [36]:
h=add.head(1500)

In [38]:
t=add.tail(1500)

In [40]:
t['pred']=0.0

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
submit=pd.concat([result,add])

In [44]:
submit.to_csv('final.txt',index=False,header=False)

### 分离特征和标签

In [19]:
features = [x for x in train.columns if x not in ['user_id', 'label',"prob",
"register_day",
"last_video_day_before_pred","last_launch_day_before_pred","last_action_day_before_pred",
# ###############video####################
#  'day_video_mean',
#  'day_video_std',
#  'day_video_max',
#  'day_video_min',
#  'day_video_skew',
#  'day_video_last',

#  'last_7_video_num',
#  'last_6_video_num',
#  'last_5_video_num',
#  'last_4_video_num',
#  'last_3_video_num',
#  'last_2_video_num',
#  'last_1_video_num',
# #  'video_day_count',
# #  'video_day_mean',
# #  'video_day_min',
# #  'video_day_max',
# #  'video_day_max-min',
#  'last_7day_video_rate',
#  'last_5day_video_rate',
#  'last_3day_video_rate',
#  'last_2day_video_rate',
#  'last_1day_video_rate',
# #  'video_day_std',
# #  'video_day_skew',
# #  'video_day_dis_max',
# #  'video_day_dis_min',
# #  'video_day_dis_mean',
# #  'video_day_dis_std',
# #  'video_day_dis_skew',
#  'video_2_continuous_day_count',
#  'video_3_continuous_day_count',
#  'video_5_continuous_day_count',
#  'video_7_continuous_day_count',
# ########################################

'action_type_1_day_dis_max', 'action_type_1_day_dis_min',
       'action_type_1_day_dis_mean', 'action_type_1_day_dis_median',
       'action_type_1_day_dis_std', 'action_type_1_day_dis_skew',
       'action_type_1_day_dis_kurt', 'action_type_1_day_dis_last',
       'action_type_1_dis_max', 'action_type_1_dis_min',
       'action_type_1_dis_mean', 'action_type_1_dis_median',
       'action_type_1_dis_std', 'action_type_1_dis_skew',
       'action_type_1_dis_kurt', 'action_type_1_dis_last',
       'action_type_2_day_dis_max', 'action_type_2_day_dis_min',
       'action_type_2_day_dis_mean', 'action_type_2_day_dis_median',
       'action_type_2_day_dis_std', 'action_type_2_day_dis_skew',
       'action_type_2_day_dis_kurt', 'action_type_2_day_dis_last',
       'action_type_2_dis_max', 'action_type_2_dis_min',
       'action_type_2_dis_mean', 'action_type_2_dis_median',
       'action_type_2_dis_std', 'action_type_2_dis_skew',
       'action_type_2_dis_kurt', 'action_type_2_dis_last',
       'action_type_3_day_dis_max', 'action_type_3_day_dis_min',
       'action_type_3_day_dis_mean', 'action_type_3_day_dis_median',
       'action_type_3_day_dis_std', 'action_type_3_day_dis_skew',
       'action_type_3_day_dis_kurt', 'action_type_3_day_dis_last',
       'action_type_3_dis_max', 'action_type_3_dis_min',
       'action_type_3_dis_mean', 'action_type_3_dis_median',
       'action_type_3_dis_std', 'action_type_3_dis_skew',
       'action_type_3_dis_kurt', 'action_type_3_dis_last',
       'action_type_4_day_dis_max', 'action_type_4_day_dis_min',
       'action_type_4_day_dis_mean', 'action_type_4_day_dis_median',
       'action_type_4_day_dis_std', 'action_type_4_day_dis_skew',
       'action_type_4_day_dis_kurt', 'action_type_4_day_dis_last',
       'action_type_4_dis_max', 'action_type_4_dis_min',
       'action_type_4_dis_mean', 'action_type_4_dis_median',
       'action_type_4_dis_std', 'action_type_4_dis_skew',
       'action_type_4_dis_kurt', 'action_type_4_dis_last',
       'action_type_5_day_dis_max', 'action_type_5_day_dis_min',
       'action_type_5_day_dis_mean', 'action_type_5_day_dis_median',
       'action_type_5_day_dis_std', 'action_type_5_day_dis_skew',
       'action_type_5_day_dis_kurt', 'action_type_5_day_dis_last',
       'action_type_5_dis_max', 'action_type_5_dis_min',
       'action_type_5_dis_mean', 'action_type_5_dis_median',
       'action_type_5_dis_std', 'action_type_5_dis_skew',
       'action_type_5_dis_kurt', 'action_type_5_dis_last',
]]
label = 'label'

###  LightGBM

In [202]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import auc, log_loss, roc_auc_score,f1_score,recall_score,precision_score
from sklearn.cross_validation import StratifiedKFold

kf = StratifiedKFold(train[label], n_folds=5, shuffle=True, random_state=1024)

params = {
            'boosting_type': 'gbdt',
            'metric': {'auc',}, 
#             'is_unbalance':'True',
            'learning_rate' : 0.01, 
             'verbose': 0,
            'num_leaves':32 ,
            # 'max_depth':8, 
            # 'max_bin':10, 
            # 'lambda_l2': 1, 
            # 'min_child_weight':50,
            'objective': 'binary', 
            'feature_fraction': 0.4,
            'bagging_fraction':0.7, # 0.9是目前最优的
            'bagging_freq':3,  # 3是目前最优的
#             'min_data': 500,
            'seed': 1024,
            'nthread': 8,
            # 'silent': True,
}
num_round = 3500
early_stopping_rounds = 100

In [219]:
train.index=range(len(train))

In [119]:
dtrain=lgb.Dataset(train[features], label=train[label])
cv_result = lgb.cv(params, dtrain, num_boost_round=4000, stratified=True,
                           nfold=5, early_stopping_rounds=100)
len(cv_result['auc-mean'])

In [120]:
best_num_boost_rounds = len(cv_result['auc-mean'])
mean_test_auc = np.mean(cv_result['auc-mean'][best_num_boost_rounds-6 : best_num_boost_rounds-1])
mean_test_auc

In [117]:
aus = []
rlt_pred = 0
for i,(train_index,test_index) in enumerate(kf):
  
    tr_x = train[features].reindex(index=train_index, copy=False).values
    tr_y = train[label][train_index]
    te_x = train[features].reindex(index=test_index, copy=False).values
    te_y = train[label][test_index]

    d_tr = lgb.Dataset(tr_x, label=tr_y)
    d_te = lgb.Dataset(te_x, label=te_y)
    model = lgb.train(params, d_tr, num_boost_round=num_round, 
                      valid_sets=d_te,verbose_eval=200,
                              early_stopping_rounds=early_stopping_rounds)
    pred = model.predict(te_x, num_iteration=model.best_iteration)
    
    te_y=te_y.apply(lambda x:1 if x>0 else 0)
    a = roc_auc_score(te_y, pred)

    t_pred = model.predict(test[features].reindex(copy=False), num_iteration=model.best_iteration)
    rlt_pred += t_pred

    print ("idx: ", i) 
    print (" auc: %.5f" % a)
#     print " gini: %.5f" % g
    print ("best tree num: ", model.best_iteration)
    aus.append(a)
rlt_pred /=5
# test['label']=test['label'].apply(lambda x:1 if x>0 else 0)
# print(roc_auc_score(test['label'], rlt_pred))
print ("mean")
print ("auc:       %s" % (sum(aus) / 5.0))


In [204]:
dtrain = lgb.Dataset(train[features], label=train[label])
# dtest=lgb.Dataset(train[features], label=train[label])
dtest = lgb.Dataset(test[features], label=test[label])
model = lgb.train(params, dtrain, num_boost_round=1400, 
                      valid_sets=[dtrain,dtest],
                  verbose_eval=100,
                early_stopping_rounds=early_stopping_rounds
             )
pred = model.predict(test[features], num_iteration=model.best_iteration)

# roc_auc_score(test[label], pred)

In [225]:
#####lgb参数扰动
# """"
# lgb随机扰动
# """
import lightgbm as lgb
import sys,random
import pickle
import os
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import auc, log_loss, roc_auc_score
import datetime

# os.mkdir('model')
# os.mkdir('preds')

def pipeline(iteration,feat_num,learn_rate,min_child,leaves,max_bin,lambda_l1,lambda_l2,feature_fraction,bagging_fraction,bagging_freq,random_seed):
      
    params = {
            'boosting_type': 'gbdt',
            'metric': {'auc',}, 
#           
            'learning_rate' : learn_rate, 
             'verbose': 0,
            'num_leaves':leaves ,
         
            'max_bin':max_bin, 
            'lambda_l1': lambda_l1, 
            'lambda_l2': lambda_l2, 
            'min_child_weight':min_child,
            'objective': 'binary', 
            'feature_fraction': feature_fraction,
            'bagging_fraction':bagging_fraction, # 0.9是目前最优的
            'bagging_freq':bagging_freq,  # 3是目前最优的
            'seed': random_seed,
            'nthread': 8,
         
}
    
    cv_num_boost_round=4000
    early_stopping_rounds=100
    cv_nfold=5
    stratified=True
    features=ff.ix[:feat_num,"feature"].values
    dtrain=lgb.Dataset(train[features], label=train[label])
    cv_result = lgb.cv(params, dtrain, num_boost_round=cv_num_boost_round, stratified=stratified,
                           nfold=cv_nfold, early_stopping_rounds=early_stopping_rounds)
    nround=len(cv_result['auc-mean'])
    print("best_nround:",nround)
    model = lgb.train(params, dtrain, num_boost_round=nround, 
                      valid_sets=[dtrain],
                  verbose_eval=200,
               
             )
    pred = model.predict(test[features])
    #predict test set
    test_result = pd.DataFrame(test['user_id'].values,columns=["user_id"])
    test_result["probility"] = pred
    test_result.to_csv("/home/kesci/input/lgb{0}.csv".format(iteration),index=None,encoding='utf-8')
    
        
if __name__ == "__main__":
    # max_bin=[230,235,240,245,250,255,260,265,270,275]
    # lambda_l1=[0.01,0.05,0.1,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
    # lambda_l2=list(range(0,20,1))
    # min_child=list(range(0,50,2))
    # leaves=[15,31,32,33,34,35,36,48,63,64,77,88,99,106,111,115,120,121,127,132]
    
    # learn_rate=[0.005,0.006,0.007,0.008,0.009,0.01,0.011,0.012,0.013,0.014,
    # 0.015,0.016,0.017,0.018,0.019,0.02]
    # feature_fraction=[i/1000 for i in range(400,710,5)]
    # bagging_fraction=[i/1000 for i in range(600,810,5)]
    # bagging_freq=[1,2,3,4,5]
    # feat_num=list(range(75,128,1))
    # random_seed =list( range(4,2066,50))
    
    # random.shuffle(feat_num)
    # random.shuffle(max_bin)
    # random.shuffle(lambda_l1)
    # random.shuffle(lambda_l2)
    # random.shuffle(min_child)
    # random.shuffle(leaves)
    # random.shuffle(learn_rate)
    # random.shuffle(feature_fraction)
    # random.shuffle(bagging_fraction)
    # random.shuffle(bagging_freq) 
    # random.shuffle(random_seed)
    
    # with open('params.pkl','wb') as f:
    #     pickle.dump((feat_num,max_bin,lambda_l1,lambda_l2,min_child,leaves,learn_rate,feature_fraction,bagging_fraction,bagging_freq,random_seed),f)
    
    with open('params.pkl','rb') as f:
        feat_num,max_bin,lambda_l1,lambda_l2,min_child,leaves,learn_rate,feature_fraction,bagging_fraction,bagging_freq,random_seed=pickle.load(f)
    
    for i in range(25,30,1):
        
        print("iter:",i)
        start=datetime.datetime.now()
        print("feat_num",feat_num[i%50],"rate:",learn_rate[i%16],"min_child:",min_child[i%25],"leaves:",leaves[i%20],"maxbin:",max_bin[i%10],"l1:",lambda_l1[i%13],"l2:",lambda_l2[i%20],"feat:",feature_fraction[i%60],"bagging:",bagging_fraction[i%40],"freq:",bagging_freq[i%5],"seed:",random_seed[i%40])
        pipeline(i,feat_num[i%50],learn_rate[i%16],min_child[i%25],leaves[i%20],max_bin[i%10],lambda_l1[i%13],lambda_l2[i%20],feature_fraction[i%60],bagging_fraction[i%40],bagging_freq[i%5],random_seed[i%40])
        print("use time:",datetime.datetime.now()-start)


#### save params:


In [229]:
# """
# 将扰动成绩汇总取平均
# """
import pandas as pd 
import os
pred = pd.read_csv('/home/kesci/input/lgb0.csv')
user_id = pred.user_id
probility = pred.probility
for f in range(1,28):
    pred = pd.read_csv('/home/kesci/input/lgb%s.csv'%str(f))
    probility += pred.probility

probility /= 28

# result=pd.DataFrame(columns=['userid','probability'])
# result.userid=test_data['id']
# result.probability=probility.values
# result.to_csv('e:/submit_lida_lgm_bagging0213.csv',sep=',',index=False)   

### feature importance

In [205]:
#####特征重要性
%matplotlib inline
import matplotlib.pyplot as plt
f=dict(zip(list(train[features].keys()),model.feature_importance()))
f=sorted(f.items(),key=lambda d:d[1], reverse = True)
f=pd.DataFrame(f,columns=['feature','imp'])
plt.bar(range(len(f)),f.imp)
plt.xticks(range(len(f)),f.feature,rotation=70,fontsize=20)
fig = plt.gcf()
fig.set_size_inches(50, 20)

In [206]:
f.ix[:220,:]

In [30]:
features=f.ix[:167,"feature"].values

### para turning 

In [ ]:
result={"best_leaves":0,"best_score":0.0}
def turning(num_leaves,train=train,test=test,result=result,params=params):
    
    params['num_leaves']=num_leaves
    dtrain = lgb.Dataset(train[features], label=train[label])
    # dtest=lgb.Dataset(train[features], label=train[label])
    dtest = lgb.Dataset(test[features], label=test[label])
    model = lgb.train(params, dtrain, num_boost_round=10000, 
                          valid_sets=[dtrain,dtest],
                      verbose_eval=10000,
                    early_stopping_rounds=early_stopping_rounds
                     )
    pred = model.predict(test[features], num_iteration=model.best_iteration)
    # test['prob']=pred
    a=roc_auc_score(test[label], pred)
    print("num_leaves:%d ==== auc:%.6f"%(num_leaves,a))
    if a>result['best_score']:
        print("###"*24)
        print("auc improved change auc:%.6f to %.6f"%(result['best_score'],a))
        print("###"*24)
        result['best_leaves']=num_leaves
        result['best_score']=a
    else:
        print("%d leaves didn't improve!!! "%(num_leaves))

for i in range(5,257):
    turning(i)

### XGBoost

In [133]:
import xgboost as xgb

params={
	'booster':'gbtree',
	'objective': 'binary:logistic',
#      'is_unbalance':'True',
# 	'scale_pos_weight': 1500.0/13458.0,
        'eval_metric': "auc",
    
	'gamma':0.1,#0.2 is ok
	'max_depth':8,
# 	'lambda':20,
    # "alpha":5,
        'subsample':0.7,
        'colsample_bytree':0.7 ,
        # 'min_child_weight':2.5, 
        'eta': 0.01,
    # 'learning_rate':0.01,
	'seed':1024,
	'nthread':12,
   
    }

dtrain = xgb.DMatrix(train[features], label=train[label])
# dval=xgb.DMatrix(train[features], label=train[label])
dval=xgb.DMatrix(test[features], label=test[label])
dtest =xgb.DMatrix(test[features])

watchlist  = [(dtrain,'train'),
    (dval,'val')
             ]
model = xgb.train(params, dtrain, num_boost_round=1200, 
                      evals=watchlist,
                  verbose_eval=200,
                  early_stopping_rounds=100
                 )
pred = model.predict(dtest)

# roc_auc_score(test[label], pred)

In [18]:
aus = []
rlt_pred = 0
for i,(train_index,test_index) in enumerate(kf):
  
    tr_x = train[features].reindex(index=train_index, copy=False)
    tr_y = train[label][train_index]
    te_x = train[features].reindex(index=test_index, copy=False)
    te_y = train[label][test_index]

    # tr_y=tr_y.apply(lambda x:1 if x>0 else 0)
    # te_y=te_y.apply(lambda x:1 if x>0 else 0)
    d_tr = xgb.DMatrix(tr_x, label=tr_y)
    d_te = xgb.DMatrix(te_x, label=te_y)
    watchlist  = [(d_tr,'train'),
    (d_te,'val')
             ]
    model = xgb.train(params, d_tr, num_boost_round=5500, 
                      evals=watchlist,verbose_eval=200,
                              early_stopping_rounds=100)
    pred = model.predict(d_te)
    
    # te_y=te_y.apply(lambda x:1 if x>0 else 0)
    a = roc_auc_score(te_y, pred)

    t_pred = model.predict(xgb.DMatrix(test[features]))
    rlt_pred += t_pred

    print ("idx: ", i) 
    print (" auc: %.5f" % a)
#     print " gini: %.5f" % g
    aus.append(a)
rlt_pred /=5
# test['label']=test['label'].apply(lambda x:1 if x>0 else 0)
# print(roc_auc_score(test['label'], rlt_pred))
print ("mean")
print ("auc:       %s" % (sum(aus) / 5.0))

#### CatBoost

In [21]:
import catboost as cat
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score,roc_auc_score

cat_model=CatBoostClassifier(
    iterations=1000,
                         learning_rate=0.1,
                         depth=8,
                        #  l2_leaf_reg=5,
                        #   eval_set=[test[features],test[label]],
                        # subsample= 0.7,
                        #  bootstrap_type='Bernoulli',
                        #  rsm=None,
                          random_seed=1024,
                        #  od_wait=100,
                        #  od_type="Iter",
                        #  use_best_model=True,
                         eval_metric="AUC",
                        logging_level= 'Silent',
                       
                        #  verbose=1,
                         loss_function='Logloss',
    )
cat_model.fit(train[features],train[label])
pred=cat_model.predict_proba(test[features])[:,1]
roc_auc_score(test[label],pred)

### NN

In [83]:
#######fix the result#######
import numpy as np
import tensorflow as tf
import random as rn
import os
import keras
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
######################
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from keras import backend as K

def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
#--------------------------------
    
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N
    
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P
    
class RocAucMetricCallback(keras.callbacks.Callback):
    def __init__(self, predict_batch_size=1024, include_on_batch=False):
        super(RocAucMetricCallback, self).__init__()
        self.predict_batch_size=predict_batch_size
        self.include_on_batch=include_on_batch
 
    def on_batch_begin(self, batch, logs={}):
        pass
 
    def on_batch_end(self, batch, logs={}):
        if(self.include_on_batch):
            logs['roc_auc_val']=float('-inf')
            if(self.validation_data):
                logs['roc_auc_val']=roc_auc_score(self.validation_data[2], 
                                                  self.model.predict([self.validation_data[0],self.validation_data[1]],
                                                                     batch_size=self.predict_batch_size))
 
    def on_train_begin(self, logs={}):
        if not ('roc_auc_val' in self.params['metrics']):
            self.params['metrics'].append('roc_auc_val')
 
    def on_train_end(self, logs={}):
        pass
 
    def on_epoch_begin(self, epoch, logs={}):
        pass
 
    def on_epoch_end(self, epoch, logs={}):
        logs['roc_auc_val']=float('-inf')
        if(self.validation_data):
            logs['roc_auc_val']=roc_auc_score(self.validation_data[2], 
                                              self.model.predict([self.validation_data[0],self.validation_data[1]],
                                                                 batch_size=self.predict_batch_size))
 
 
import numpy as np
from keras.models import Sequential
from keras.layers import GRU,LSTM,Input,Dense, Dropout,BatchNormalization,Concatenate,Embedding
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.metrics import roc_auc_score
from keras import metrics

# X_train=train[features].fillna(0)
# X_test=test[features].fillna(0)
# X_train = np.reshape(X_train.values, (len(train), 16, 1))
# X_test = np.reshape(X_test.values, (len(test), 16, 1))
# X_train=np.array(tr.iloc[:,2:18])
# X_test=np.array(te.iloc[:,2:18])
X_train=tr.iloc[:,2:18].values
X_test=te.iloc[:,2:18].values
feat_train=train[features].fillna(-1).values
feat_test=test[features].fillna(-1).values
y_train=train[label].values
y_test=test[label].values
model_checkpoint = ModelCheckpoint("/home/kesci/input/nn.h5", monitor='roc_auc_val', mode='max',
                                       save_best_only=True, verbose=1, save_weights_only=True)

cb = [
    RocAucMetricCallback(), # include it before EarlyStopping!
     model_checkpoint,
    EarlyStopping(monitor='roc_auc_val',patience=10, verbose=1,mode='max')
]
input_tensor=Input(shape=(16,))
emb=Embedding(2, 64 , input_shape=(16,),
                         trainable=True)(input_tensor)
lstm_layer =LSTM(32,activation='tanh',recurrent_dropout=0.2)
lstm=lstm_layer(emb)

lstm = Dropout(0.2)(lstm)
feat_input = Input(shape=(128,))
feat_dense = Dense(128, activation='relu')(feat_input)
merged = Concatenate()([lstm,feat_dense])
merged = Dropout(0.2)(merged)
merged = BatchNormalization()(merged)
x = Dense(64, activation='relu')(merged)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x)
ouput_layer = Dense(1, activation='sigmoid')(x)

model = keras.models.Model(inputs=[input_tensor,feat_input], outputs=ouput_layer)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[auc]) #这里就可以写其他评估标准
model.summary()

train[label]=train[label].astype(int)
test[label]=test[label].astype(int)
model.fit(x=[X_train,feat_train],
        y=y_train,
         validation_data=([X_test,feat_test],y_test),
        batch_size=1024, epochs=1500, verbose=1,
              callbacks=cb,
         
          )

In [84]:
model.load_weights("/home/kesci/input/nn.h5")
# pred = model.predict(np.reshape(test[features].fillna(-1).values,(len(test),71,1)), batch_size=2048, verbose=1)
pred = model.predict([X_test,test[features].fillna(-1).values], verbose=1)
roc_auc_score(test[label],pred)

#### TIME SEQUENCE FEATURE

In [11]:
from collections import Counter
count=0
def get_time_feat(df,pred_firstday):
    global count
    count+=1
    df['day']=df.day.astype("str")
    df['day']=df.day.apply(lambda x:pred_firstday+"_"+str(x),1)
    temp=pd.DataFrame(columns=[pred_firstday+"_"+str(i) for i in range(1,31)],index=range(1))
    temp["user_id"]=df['user_id'].values[-1]
    
    l=list(df['day'].values)
    day_list=Counter(l)
    if len(day_list.keys())==1:
        temp.loc[0,list(day_list.keys())]=list(day_list.values())[0]
    else:
        temp.loc[0,list(day_list.keys())]=list(day_list.values())
    if count%10000==0:
        print(count)
    return temp

In [12]:
launch_feat=applyParallel(get_time_feat,launch,"action")

In [28]:
temp=launch_feat.iloc[:,0:16]
temp['user_id']=launch_feat['user_id']
tr=train[['user_id',"label"]]
tr=pd.merge(tr,temp)

In [31]:
temp=launch_feat.iloc[:,7:23]
temp['user_id']=launch_feat['user_id']
te=test[['user_id','label']]
te=pd.merge(te,temp)

## stacking

In [ ]:
from __future__ import division

import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation, linear_model

# MAX_ITERS = 1
# N_TREES = 5
# XGBC_TREES = 5
# LM_CV_NUM = 5

N_TREES = 500
XGBC_TREES = 1500
LM_CV_NUM = 100
MAX_ITERS = 400

INITIAL_PARAMS = {
    'LR:one': {'C': 2, 'penalty': 'l2','class_weight':"balanced",  'n_jobs': -1,
               'max_iter': MAX_ITERS,
               },
    'XGBC:one': {
        'n_estimators': XGBC_TREES,
        'scale_pos_weight': 6,
        'objective': 'binary:logistic',
        'learning_rate': 0.02,
        'gamma': 0.7,
        'reg_lambda': 800,
        'colsample_bytree': 0.7,
        'max_depth': 5,
        'min_child_weight': 4,
        'subsample': 0.7,
        },
    'XGBC:two': {
        'n_estimators': 400,
        'scale_pos_weight': 1,
        'objective': 'binary:logistic',
        'learning_rate': 0.04,
        'colsample_bytree': 0.7,
        'max_depth': 5,
        'subsample': 0.7,
        },
    'XGBC:three':{
        'objective':'binary:logistic',
        'n_estimators': 1500,
      'min_child_weight':1,
        
        "reg_lambda":11,
         "reg_alpha":3,
            'scale_pos_weight': 1,
#             'is_unbalance':'True',
            'learning_rate' : 0.01, 
             
        'gamma':0.1,
        'max_depth':5,
         'subsample': 0.68,
        'colsample_bytree': 0.68,
        
        'seed':1024,
         "random_state":1024,
	'nthread':7,
        
    },
     "LightGBM":{
            'boosting_type': 'gbdt',
             'n_estimators': 1500,
            'metric': 'auc', 
            'is_unbalance':True,
            'learning_rate' : 0.01, 
             'verbose': 0,
          'min_child_weight':1,
            
            'max_depth':5, 
            'max_bin':15, 
          "reg_lambda":11,
         "reg_alpha":3,
            'scale_pos_weight': 1,
            'objective': 'binary', 
            'colsample_bytree': 0.68,
            'subsample':0.68, # 0.9是目前最优的
            'subsample_freq':1,  # 3是目前最优的
            'min_data': 500,
            'seed': 1024,
         
            'nthread': 12,
#             'silent': True,
},
    
    
      
       
       
     
       
       
    'RFC:one': {
        # 'n_estimators': N_TREES, 'n_jobs': -1, 'criterion': 'entropy',
        # 'min_samples_leaf': 2, 'bootstrap': False,
        # 'max_depth': 20, 'min_samples_split': 7, 'max_features': 0.5
        # new tune
        'n_estimators': 400, 'n_jobs': -1, 'criterion': 'entropy',
        'min_samples_leaf': 3, 'bootstrap': False,
        'max_depth': 12, 'min_samples_split': 6, 'max_features': 0.14357
    },
    'ETC:one': {
        'n_estimators': N_TREES, 'n_jobs': -1, 'min_samples_leaf': 2,  'criterion': 'entropy',
        'max_depth': 20, 'min_samples_split': 5, 'max_features': 0.4,
        'bootstrap': False,
        },
    'GBC:one': {
        'n_estimators': int(N_TREES / 2), 'learning_rate': .08, 'max_features': 0.7,
        'min_samples_leaf': 51, 'min_samples_split': 125, 'max_depth': 4,
    },
    }

import xgboost as xgb
import lightgbm as lgb
MODEL_NAME = 'blend_ensemble'


def blending(X_org, y_org, test_org,on_val=False,use_single_model=False):
    n_folds =5
    verbose = True
    shuffle = False

    X = X_org
    y = y_org
    X_submission = test_org[features]
    
    if shuffle:
        idx = np.random.permutation(y.size)
        X = X[idx]
        y = y[idx]

    skf = list(StratifiedKFold(y, n_folds))

    clfs = [
        RandomForestClassifier().set_params(**INITIAL_PARAMS.get("RFC:one", {})),
        ExtraTreesClassifier().set_params(**INITIAL_PARAMS.get("ETC:one", {})),
        GradientBoostingClassifier().set_params(**INITIAL_PARAMS.get("GBC:one", {})),
#         LogisticRegression().set_params(**INITIAL_PARAMS.get("LR:one", {})),
        xgb.XGBClassifier().set_params(**INITIAL_PARAMS.get("XGBC:two", {})),
        xgb.XGBClassifier().set_params(**INITIAL_PARAMS.get("XGBC:one", {})),
        lgb.LGBMClassifier().set_params(**INITIAL_PARAMS.get("LightGBM", {})),
         xgb.XGBClassifier().set_params(**INITIAL_PARAMS.get("XGBC:three", {})),

        ]


    print ("Creating train and test sets for blending.")

    dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
    dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))
    for j, clf in enumerate(clfs):
        print (j, clf)
        dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
        
        for i, (train, test) in enumerate(skf):
            print ("Fold", i)
            
            X_train = X.reindex(index=train, copy=False)
            y_train = y[train]
            X_test = X.reindex(index=test, copy=False)
            y_train=y_train.apply(lambda x:1 if x>0 else 0)
            X_train=X_train.fillna(0)
            X_test=X_test.fillna(0)
            y_test = y[test]
            clf.fit(X_train, y_train)
            y_submission = clf.predict_proba(X_test)[:,1]
            dataset_blend_train[test, j] = y_submission
            dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:,1]
       
        dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)
        if on_val:
            print("val auc Score: %.10f" % (roc_auc_score(test_org.label, dataset_blend_test[:,j]))) 

    if use_single_model:
        return dataset_blend_test
    print ("Blending.")
   # clf = LogisticRegression(C=2, penalty='l2', class_weight='balanced', n_jobs=-1)
    clf = linear_model.RidgeCV(
        alphas=np.linspace(0, 200), cv=LM_CV_NUM)
    # clf = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=100)
    clf.fit(dataset_blend_train, y)
    # y_submission = clf.predict_proba(dataset_blend_test)[:,1]
#     print clf.coef_, clf.intercept_
    y_submission = clf.predict(dataset_blend_test)  # for RidgeCV
    
    print ("Linear stretch of predictions to [0,1]")
    y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
    if on_val:
        print("********4****val auc Score: %0.5f" % (roc_auc_score(test_org.label, y_submission)))
        df=pd.DataFrame(dataset_blend_test,columns=["m1","m2","m3","m4","m5","m6"])
        df["prob"]=df['m1']*0.1+df["m2"]*0.1+df['m3']*0.1+df["m4"]*0.2+df['m5']*0.3+df["m6"]*0.2
        print("val auc Score: %0.5f" % (roc_auc_score(test_org.label,df['prob'] )))
    return y_submission
import time
start=time.time()
y_submission=blending(train[features],train[label],test,on_val=True,use_single_model=True)
end=time.time()
print("time:%.2f s"%(end-start))
print ("blend result")
# save(test, y_submission, "xgb")


## bagging

In [1]:
import pandas as pd
result1=pd.read_csv("bagging28.txt",header=None)
result1.columns=['user_id',"pred"]
result2=pd.read_csv("bagging.txt",header=None)
result2.columns=['user_id',"pred"]
# result3=pd.read_csv("cat.txt",header=None)
# result3.columns=['user_id',"pred"]

In [3]:
result1[result1['user_id']==547]

In [31]:
result1['prob']=result1["pred"]*0.6+0.4*result2["pred"]

In [50]:
result1[['user_id',"prob"]].to_csv("0.6+0.4.txt",index=False,header=False)

In [55]:
test['prob1']=result1['pred']
test['prob2']=result2['pred']
# test['prob4']=pred
# test['prob3']=result3['pred']

In [57]:
#归一化
min1= test['prob1'].min()
max1= test['prob1'].max()
test['prob1'] = test['prob1'].map(lambda x:(x-min1)/(max1-min1))

min2= test['prob2'].min()
max2= test['prob2'].max()
test['prob2'] = test['prob2'].map(lambda x:(x-min2)/(max2-min2))

# min3= test['prob3'].min()
# max3= test['prob3'].max()
# test['prob3'] = test['prob3'].map(lambda x:(x-min3)/(max3-min3))

# min5= test['prob4'].min()
# max5= test['prob4'].max()
# test['prob4'] = test['prob4'].map(lambda x:(x-min5)/(max3-min5))

## 简单加权融合
submit=pd.DataFrame()
submit['user_id']=test.user_id
submit['prob'] = 0.5*test['prob1'] + 0.5*test['prob2']

min4= submit['prob'].min()
max4= submit['prob'].max()
submit['prob'] = submit['prob'].map(lambda x:(x-min4)/(max4-min4))
submit.to_csv('bagging.txt',index=False,header=False)

### output

In [230]:

def save(test, pred, name):
    test['prob']=pred
    test[['user_id','prob']].to_csv("%s.txt" % ( name), index=False,header=False)
    
    

In [232]:
save(test,probility,"bagging28")

In [50]:
# 下载提交工具至当前目录，仅需执行一次
# !wget -nv -O kesci_submit https://cdn.kesci.com/submit_tool/v1/kesci_submit&&chmod +x kesci_submit
# 提交文件my_submission.txt进行评审；温馨提示:本次比赛提交文件的格式为txt
!./kesci_submit -token 18c6e182512f3cfb -file final.txt

Kesci Submit Tool
Result File: final.txt (14.84 MiB)
Uploaded.       
Submit Success.
OK

